In [0]:
import numpy as np
import pandas as pd

In [0]:
df_usetrucksales_raw = spark.sql("SELECT * FROM df_use_truck_sales_new").toPandas()
df_construction_detail_raw = spark.sql("SELECT * FROM equipment_configuration_table_new").toPandas() 
df_operation_hours_raw = spark.sql("SELECT * FROM df_operating_hours_new").toPandas()
df_truckclass = spark.sql("SELECT * FROM truckclass_table_new").toPandas()
config_detail_df = spark.sql('SELECT * FROM config_detail_new where SPRAS = "E"').toPandas()

In [0]:
# User Input
CONTRACT_TYPES_OF_INTEREST = ['Z201', 'Z2L1', 'Z281', 'Z298', 'Z3L9', 'Z309', 'Z381', 'Z398', 'Z2PN']
INVOICE_TYPES_OF_INTEREST = ['Z201', 'Z202', 'Z281', 'Z2S1', 'Z3S3', 'Z309', 'Z381', 'Z3S5', 'Z3S7']
MATERIAL_NR_OF_INTEREST = df_truckclass[df_truckclass.CLASS !='ACC'].MATNR.unique() 

In [0]:
def preprocess_uts(uts_df):
  # Define Variables
  uts_df['FINANCIAL_VEHICLE_ID'] = uts_df.OrderNr
  uts_df['EQUNR'] = '00000000' +  uts_df.Charge
  uts_df['SOLD_PRICE'] = uts_df.InvoicePositionValue_EURO 
  uts_df['SOLD_DATE'] = pd.to_datetime(uts_df['InvoiceDate']) 
  uts_df['SOLD_YEAR'] = uts_df.SOLD_DATE.dt.year.astype(int)
  uts_df['DELIVERY_DATE'] = pd.to_datetime(uts_df['DeliveryDate']) 
  uts_df['STATUS'] = np.where(uts_df.Status.str.len() == 0, '7', uts_df.Status.fillna('7')).astype(int)
  uts_df['TRADER_ID'] = uts_df.BranchOwner
  uts_df['Invoice_Nr_Numeric'] = uts_df.InvoiceNr.astype(int)
  uts_df['MATERIAL_NR'] = uts_df.MaterialNr
  uts_df['SOLD_COUNTRY'] = uts_df.countryCode_we
  
  uts_df['CUSTOMER_TYPE_COMBINED_COLUMN_TMP'] = uts_df[[x for x in uts_df.columns if "Customer_Type" in x]].astype(str).sum(axis=1).str.replace(' ', '')
  uts_df['CUSTOMER_TYPE'] = np.where((uts_df.CUSTOMER_TYPE_COMBINED_COLUMN_TMP.str.contains('Dealer') | uts_df.CUSTOMER_TYPE_COMBINED_COLUMN_TMP.str.contains('Trader')), 'Trader', 'End-Customer')
  uts_df['LEASING_FLAG'] = np.where(uts_df.CUSTOMER_TYPE_COMBINED_COLUMN_TMP.str.contains('Leasing'), 1, 0)
  uts_df['KEY_ACCOUNT_FLAG'] = np.where(uts_df.KeyAccountTag_we.str.len() > 0, 1, 0)
  uts_df = uts_df.drop(columns=['CUSTOMER_TYPE_COMBINED_COLUMN_TMP'])
  return uts_df
 
def preprocess_and_add_operating_hour_information(df_uts, df_operating_hours):
  df_operating_hours['OPERATING_HRS_CALCULATED'] = df_operating_hours.OPERATING_S // 3600.
  df_operating_hours['OPERATING_HRS_DIFFERENCE'] = df_operating_hours.OPERATING_HRS - df_operating_hours.OPERATING_HRS_CALCULATED
  df_operating_hours['OPERATING_HOURS'] = np.where(df_operating_hours.OPERATING_HRS >= df_operating_hours.OPERATING_HRS_CALCULATED, df_operating_hours.OPERATING_HRS, df_operating_hours.OPERATING_HRS_CALCULATED)
  df_operating_hours = df_operating_hours.dropna(subset=['OPERATING_HOURS']).sort_values('OPERATING_HOURS').query('OPERATING_HOURS < 1000000 and OPERATING_HOURS > 0')
  df_operating_hours['MEASURE_DATE'] = pd.to_datetime(df_operating_hours.MEASURE_DATE)
  df_operating_hours = df_operating_hours[['EQUNR', 'MEASURE_DATE', 'OPERATING_HOURS']]
  
  df_operating_hours = df_operating_hours.sort_values(['MEASURE_DATE']).reset_index(drop=True)
  df_uts = df_uts.sort_values(['SOLD_DATE']).reset_index(drop=True)
  
  return pd.merge_asof(df_uts, df_operating_hours, by=['EQUNR'], left_on=['SOLD_DATE'], right_on=['MEASURE_DATE'])
 
def preprocess_and_add_construction_information(df_uts, df_construction_info):
  df_construction_info = df_construction_info[['EQUNR', 'ERDAT', 'BAUJJ', 'BAUMM']] 
  df_construction_info = df_construction_info.rename(columns={'ERDAT': 'ORDER_YEAR', 'BAUJJ':'CONSTRUCTION_YEAR', 'BAUMM':'CONSTRUCTION_TYPE'})
  df_uts = df_uts.merge(df_construction_info, on=['EQUNR'], how='left')
  df_uts.CONSTRUCTION_YEAR = df_uts.CONSTRUCTION_YEAR.fillna('-1').astype(int)
  df_uts['AGE'] = df_uts.SOLD_DATE.dt.year.astype(int) - df_uts.CONSTRUCTION_YEAR
  return df_uts
 
def clean_dataset(df_target):
  df_target_cleaned = df_target.copy()
 
  print('**** Cleaning DataSet  *******')
  print('Number of Sales Doc in UTS Raw:', df_target.shape[0])
  print('Number of Orders in UTS Raw:', len(df_target.OrderNr.unique()))
  number_of_orders_left = len(df_target.OrderNr.unique())
 
  # Filter Contract Types Of Interest
  print('\n UTS Cleaning I Contract Types , Invoice Types and Material Types (exclude ACC-Class)')
  df_target_cleaned = df_target_cleaned[df_target_cleaned.ContractType.isin(CONTRACT_TYPES_OF_INTEREST) &  df_target_cleaned.InvoiceType.isin(INVOICE_TYPES_OF_INTEREST)]
  df_target_cleaned = df_target_cleaned[df_target_cleaned.MaterialNr.isin(MATERIAL_NR_OF_INTEREST)] #TODO: Check Problem Battery in same order without Price?
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
  
  print('\n UTS Preprocessing II: Keep only last Status for each Fiancial Vehicle')
  max_invoice_number_per_order_df = df_target_cleaned.groupby(['FINANCIAL_VEHICLE_ID']).Invoice_Nr_Numeric.max().reset_index()
  df_target_cleaned = df_target_cleaned.merge(max_invoice_number_per_order_df, on=['FINANCIAL_VEHICLE_ID', 'Invoice_Nr_Numeric'])
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
  
  print('\n UTS Preprocessing III : Filter Out Consolidation Entries as last status')
  df_target_cleaned = df_target_cleaned[df_target_cleaned.ContractCategory != "Consolidation"].reset_index(drop=True)
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
  
  print('\n UTS Preprocessing IV : Filter Out Quantity -1 as last status of Fiancial Vehicle')
  df_target_cleaned = df_target_cleaned[df_target_cleaned.Quantity > 0].reset_index(drop=True)
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
  
  print('\n UTS Preprocessing V : Filter Orders of EQUNR which are sold in the next 365 days again:')
  df_target_cleaned.InvoiceDate = pd.to_datetime(df_target_cleaned.InvoiceDate)
  df_target_cleaned = df_target_cleaned.sort_values(['EQUNR', 'InvoiceDate']).reset_index(drop=True)
  number_sellings_per_equnr = df_target_cleaned.groupby('EQUNR').InvoiceNr.count().reset_index().rename(columns={'InvoiceNr': 'NUMBER_SALES_OF_EQUNR'})
  df_target_cleaned = df_target_cleaned.merge(number_sellings_per_equnr, on='EQUNR')
 
  df_target_cleaned['NEXT_INVOICE_DATE'] = df_target_cleaned.groupby('EQUNR').InvoiceDate.shift(-1)
  df_target_cleaned['OFFSET_TO_NEXT_INVOICE'] = (df_target_cleaned.NEXT_INVOICE_DATE - df_target_cleaned.InvoiceDate).astype('timedelta64[D]')
  number_docs_before = df_target_cleaned.shape[0]
  df_target_cleaned =  df_target_cleaned[(df_target_cleaned.OFFSET_TO_NEXT_INVOICE.isnull()) | (df_target_cleaned.OFFSET_TO_NEXT_INVOICE > 365)].reset_index(drop=True)
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
  
  print('\n UTS Preprocessing VI : Filter Out Vehicles without Status Information')
  df_target_cleaned = df_target_cleaned[df_target_cleaned.STATUS < 7].reset_index(drop=True)
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
 
  print('\n UTS Preprocessing VII : Filter Out Vehicles with 0 Price')
  df_target_cleaned = df_target_cleaned[df_target_cleaned.SOLD_PRICE > 0].reset_index(drop=True)
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
  
  
  print('\n UTS Preprocessing VIII : Filter Out Old Vehicles (>= 20 Years)')
  df_target_cleaned = df_target_cleaned[(df_target_cleaned.AGE < 20) & (df_target_cleaned.AGE > 0) ].reset_index(drop=True)
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
  
  print('\n UTS Preprocessing IX: Keep Only "End-Customer" CUSTOMER_TYPE ')
  df_target_cleaned = df_target_cleaned[df_target_cleaned.CUSTOMER_TYPE == "End-Customer"].reset_index(drop=True)
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
     
 
  print('\nFiltering Sales Docs without Operating Hour Information')
  number_docs_before = df_target_cleaned.shape[0]
  df_target_cleaned =  df_target_cleaned.query('OPERATING_HOURS == OPERATING_HOURS')
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
    
  print('\nFiltering Sales Docs with outdated Operating Hour Information (older than 365 days)')
  number_docs_before = df_target_cleaned.shape[0]
  df_target_cleaned = df_target_cleaned[(df_target_cleaned.SOLD_DATE - df_target_cleaned.MEASURE_DATE).astype('timedelta64[D]')<365]
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
  
  print('\nFilter Sales Docs with too small Operating Hours (<100)')
  number_docs_before = df_target_cleaned.shape[0]
  df_target_cleaned = df_target_cleaned.query('OPERATING_HOURS >= 100').reset_index(drop=True)
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
 
  print('\nFilter Sales Docs without CONSTRUCTION YEAR Information')
  number_docs_before = df_target_cleaned.shape[0]
  df_target_cleaned =  df_target_cleaned.query('CONSTRUCTION_YEAR > 0').reset_index(drop=True)
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
  
  print('\nFILTER Old Sales Docs (< 2013)')
  number_docs_before = df_target_cleaned.shape[0]
  df_target_cleaned =  df_target_cleaned[df_target_cleaned.SOLD_DATE.dt.year >= 2013].reset_index(drop=True)
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
  
  print('\nFilter Materials with few counts (<100)')
  number_docs_before = df_target_cleaned.shape[0]
  materials_valid_counts = df_target_cleaned.groupby('MaterialNr').EQUNR.count().reset_index().query('EQUNR> 100').MaterialNr.unique()
  print('Number Materials fullfilling condition', len(materials_valid_counts))
  df_target_cleaned = df_target_cleaned[df_target_cleaned.MaterialNr.isin(materials_valid_counts)]
  number_of_orders_left_prev = number_of_orders_left
  number_of_orders_left = len(df_target_cleaned.OrderNr.unique())
  print('Number of Records:', df_target_cleaned.shape[0], '\t Number of Financial Vehicles', number_of_orders_left, ' Lost Vehicles:', number_of_orders_left_prev - number_of_orders_left )
 
  return df_target_cleaned
 
def finalize_dataset(df_taget):
  number_sellings_per_invoice = df_taget.groupby('InvoiceNr').FINANCIAL_VEHICLE_ID.count().reset_index().rename(columns={'FINANCIAL_VEHICLE_ID':'NUMBER_TRUCKS_ON_INVOICE'})
  condtions = [number_sellings_per_invoice.NUMBER_TRUCKS_ON_INVOICE == 1, number_sellings_per_invoice.NUMBER_TRUCKS_ON_INVOICE == 2, number_sellings_per_invoice.NUMBER_TRUCKS_ON_INVOICE <= 10, number_sellings_per_invoice.NUMBER_TRUCKS_ON_INVOICE <= 25]
  results = ['single', 'double', 'small', 'medium']
  number_sellings_per_invoice['SOLD_PACKAGE_SIZE'] = np.select(condtions, results, default='large')
  df_taget = df_taget.merge(number_sellings_per_invoice, on=['InvoiceNr'])
  return df_taget
 

def preprocess_and_add_class_information(df_target, df_classinfos):
  df_target = df_target.merge(df_classinfos[['MATNR', 'CAPACITY', 'WHEEL_QT']], left_on='MaterialNr', right_on='MATNR', how='left')
  df_target = df_target.drop(columns=['MATNR'])
  return df_target
 
 
 
def preprocess_and_add_config_information(df_target, df_config):
  # First Mast Type
  mast_type_information_df = config_detail_df[config_detail_df.ATBEZ == "Mast type"][['EQUNR', 'ATWRT']].rename(columns={'ATWRT': 'MAST_TYPE'})
  mast_type_information_df.MAST_TYPE  = mast_type_information_df.MAST_TYPE.str.replace('TRILEX', 'T').str.replace('dreifach', 'T').str.replace('Dreifach', 'T')
  mast_type_information_df.MAST_TYPE = np.where(mast_type_information_df.MAST_TYPE.isin(['T', 'D', 'N', 'V', 'S']), mast_type_information_df.MAST_TYPE, 'O')
  df_target = df_target.merge(mast_type_information_df, how='left', on='EQUNR')
  df_target.MAST_TYPE = df_target.MAST_TYPE.fillna('U')
  
  # Second Mast HEIGHT
  mast_height_information_df = df_config[df_config.ATBEZ == "H1 Mast closed height"].rename(columns={'ATFLV': 'MAST_HEIGHT_RAW'})
  mast_height_information_df.MAST_HEIGHT_RAW = np.where(mast_height_information_df.MAST_HEIGHT_RAW >= 20000, mast_height_information_df.MAST_HEIGHT_RAW / 1000., mast_height_information_df.MAST_HEIGHT_RAW )
  mast_height_information_df['MAST_HEIGHT'] = mast_height_information_df.MAST_HEIGHT_RAW // 100
  mast_height_information_df = mast_height_information_df.sort_values('MAST_HEIGHT').drop_duplicates(subset=['EQUNR'], keep='last')
  df_target = df_target.merge(mast_height_information_df[['EQUNR', 'MAST_HEIGHT']], on='EQUNR', how='left')  
  df_target.MAST_HEIGHT = df_target.MAST_HEIGHT.fillna(-1)
 
  return df_target

In [0]:
target_df = preprocess_uts(df_usetrucksales_raw.query('DeliveryDate != "Kion Group"').copy())
target_df = preprocess_and_add_operating_hour_information(target_df, df_operation_hours_raw)
target_df = preprocess_and_add_construction_information(target_df, df_construction_detail_raw)
target_df = preprocess_and_add_class_information(target_df, df_truckclass)
target_df = preprocess_and_add_config_information(target_df, config_detail_df)
target_df = clean_dataset(target_df)
target_df = finalize_dataset(target_df)

In [0]:
target_df = target_df[target_df["MaterialNr"].str.contains("G_RX20")==True]


In [0]:
features =["AGE","CUSTOMER_TYPE","OPERATING_HRS","MATERIAL_NR","SOLD_PRICE","SOLD_YEAR","SOLD_COUNTRY","STATUS","CONSTRUCTION_YEAR","MAST_TYPE","MAST_HEIGHT","SOLD_PACKAGE_SIZE","FINANCIAL_VEHICLE_ID","CAPACITY","WHEEL_QT","BLACK_FORX_FLAG","KEY_ACCOUNT_FLAG","LEASING_FLAG"]

In [0]:
target_df.to_csv('../../dbfs/mnt/nereva_2/rx20_all_models.csv', index=False)